# Denormalisation schema 2

**Goal :**

1) Actors

2) Movies

3) Directors

## Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
from tqdm import tqdm

## JSON files

In [2]:
path = os.path.join(os.getcwd(), "IMDB\json_files")
# directory (CWD) 
print("Current working directory:", path) 

Current working directory: C:\Users\qnava\Desktop\Ecole\Cours\A5\S9\Structures de données pour le cloud\Projet\IMDB\json_files


In [3]:
Directors = pd.read_json(path+"\directors.json")
Directors

,id,first_name,last_name
0,1,Todd,1
1,2,Les,12 Poissons
2,3,Lejaren,a'Hiller
3,4,Nian,A
4,5,Khairiya,A-Mansour
...,...,...,...
86875,88797,Yusuf,Ünal
86876,88798,Ahmet,Ündag
86877,88799,Idil,Üner
86878,88800,Yüksel,Ünsal


In [4]:
Directors_genres = pd.read_json(path+"\directors_genres.json").sort_values(by="director_id")
Directors_genres

,director_id,genre,prob
0,2,Short,1.0
1,3,Drama,1.0
2,5,Documentary,1.0
3,6,Drama,1.0
4,6,Short,1.0
...,...,...,...
156557,88797,Drama,1.0
156558,88798,Adventure,1.0
156559,88799,Short,1.0
156560,88800,Animation,1.0


In [5]:
Directors_genres.groupby(["genre"]).agg("count").mean()

director_id    7828.1
prob           7828.1
dtype: float64

In [6]:
Movies = pd.read_json(path+"\movies.json")
Movies

,id,name,year,rank
0,0,#28,2002,NaN
1,1,"#7 Train: An Immigrant Journey, The",2000,NaN
2,2,$,1971,6.4
3,3,"$1,000 Reward",1913,NaN
4,4,"$1,000 Reward",1915,NaN
...,...,...,...,...
388264,412316,"""zem blch krlu""",1991,NaN
388265,412317,"""rgammk""",1995,NaN
388266,412318,"""zgnm Leyla""",2002,NaN
388267,412319,""" Istanbul""",1983,NaN


In [18]:
Movies_genres2= Movies_genres.set_index("movie_id").join(Movies.set_index("id"))

In [25]:
Movies_genres2.drop_duplicates(["genre","name"]).groupby("genre").agg("count")["name"].mean()

18057.95238095238

In [22]:
Movies_genres2.head()

,genre,name,year,rank
1,Documentary,"#7 Train: An Immigrant Journey, The",2000,NaN
1,Short,"#7 Train: An Immigrant Journey, The",2000,NaN
2,Comedy,$,1971,6.4
2,Crime,$,1971,6.4
5,Western,"$1,000 Reward",1923,NaN


In [7]:
Movies_genres = pd.read_json(path+"\movies_genres.json")
Movies_genres

,movie_id,genre
0,1,Documentary
1,1,Short
2,2,Comedy
3,2,Crime
4,5,Western
...,...,...
395114,378612,Adventure
395115,378612,Drama
395116,378613,Comedy
395117,378613,Drama


In [17]:
Movies_genres.groupby("genre").agg("count").mean()

movie_id    18815.190476
dtype: float64

In [8]:
#Nb films par genre : 
Movies_genres.groupby("genre").count().mean()

movie_id    18815.190476
dtype: float64

In [9]:
Movies_genres["genre"].unique()

array(['Documentary', 'Short', 'Comedy', 'Crime', 'Western', 'Family',
       'Animation', 'Drama', 'Romance', 'Mystery', 'Thriller', 'Adult',
       'Music', 'Action', 'Fantasy', 'Sci-Fi', 'Horror', 'War', 'Musical',
       'Adventure', 'Film-Noir'], dtype=object)

In [10]:
Movies_directors = pd.read_json(path+"\movies_directors.json").sort_values(by="movie_id")
Movies_directors

,director_id,movie_id
163516,38057,0
257818,60728,1
327612,77912,1
41291,9970,2
303093,71885,5
...,...,...
317133,75198,412317
182167,42605,412317
3040,890,412318
339033,80751,412319


In [11]:
Roles = pd.read_json(path+"\hroles.json")
Roles

,actor_id,movie_id,role
0,2,280088,Stevie
1,2,396232,Various/lyricist
2,3,376687,Gitano 1
3,4,336265,El Cigala
4,5,135644,Himself
...,...,...,...
3431961,845461,137097,Kata
3431962,845462,208838,Magga
3431963,845463,870,Gunna
3431964,845464,378123,Gudrun


In [12]:
Actors = {}
with open(path+"\hactors.json","r",encoding="utf8") as f:
    Actors = json.load(f)


In [13]:
path+"\hactors.json"

'C:\\Users\\qnava\\Desktop\\Ecole\\Cours\\A5\\S9\\Structures de données pour le cloud\\Projet\\IMDB\\json_files\\hactors.json'

In [14]:
Actors=  pd.read_json(path+"\hactors.json")

In [15]:
Directors.head()

,id,first_name,last_name
0,1,Todd,1
1,2,Les,12 Poissons
2,3,Lejaren,a'Hiller
3,4,Nian,A
4,5,Khairiya,A-Mansour


In [16]:
Directors.groupby(["first_name","last_name"]).agg('count')["id"].mean()

1.0001036018924612

## 1) Actors

In [11]:
for actor in tqdm(Actors):
    actor["roles"] = []
    temp = Roles[Roles["actor_id"]==actor["id"]]
    if(temp.shape[0] >=1):
        for row in range(temp.shape[0]):
            dico = {"role":str(temp.iloc[row]["role"]),"movie_id":int(temp.iloc[row]["movie_id"])}
            actor["roles"].append(dico)

100%|██████████| 817718/817718 [55:36<00:00, 245.08it/s]  


In [15]:
Actors[:5]

[{'id': 2,
  'first_name': 'Michael',
  'last_name': "'babeepower' Viera",
  'gender': 'M',
  'roles': [{'role': 'Stevie', 'movie_id': 280088},
   {'role': 'Various/lyricist', 'movie_id': 396232}]},
 {'id': 3,
  'first_name': 'Eloy',
  'last_name': "'Chincheta'",
  'gender': 'M',
  'roles': [{'role': 'Gitano 1', 'movie_id': 376687}]},
 {'id': 4,
  'first_name': 'Dieguito',
  'last_name': "'El Cigala'",
  'gender': 'M',
  'roles': [{'role': 'El Cigala', 'movie_id': 336265}]},
 {'id': 5,
  'first_name': 'Antonio',
  'last_name': "'El de Chipiona'",
  'gender': 'M',
  'roles': [{'role': 'Himself', 'movie_id': 135644}]},
 {'id': 6,
  'first_name': 'José',
  'last_name': "'El Francés'",
  'gender': 'M',
  'roles': [{'role': '', 'movie_id': 12083}]}]

In [13]:
with open('Structure2\ActorFinal.json', 'w') as f:
    json.dump(Actors, f)

In [23]:
Movie_genres

NameError: name 'Movie_genres' is not defined

## 2) Movies

In [14]:
A = Movies_genres.groupby(by="movie_id")["genre"].apply(list)

In [15]:
B = Movies_directors.groupby(by= "movie_id").director_id.apply(list)

In [16]:
C = Roles.groupby(by = "movie_id").actor_id.apply(list)

In [17]:
M = Movies.set_index("id").join([A,B,C])

In [18]:
x=[""]

In [19]:
M["genre"]

id
0                          NaN
1         [Documentary, Short]
2              [Comedy, Crime]
3                          NaN
4                          NaN
                  ...         
412316                     NaN
412317                     NaN
412318                     NaN
412319                     NaN
412320                     NaN
Name: genre, Length: 388269, dtype: object

In [24]:
M=M.reset_index()

In [25]:
result = M.to_json(orient="records")
parsed = json.loads(result)

In [43]:
result = M.to_json(orient="records")
parsed = json.loads(result)

In [27]:
with open('Structure2\MoviesFinal.json', 'w') as f:
    json.dump(parsed, f)

## 3) Directors

In [9]:
C = Movies_directors.groupby(by= "director_id").movie_id.apply(list)

In [10]:
C.head()

director_id
1           [378879]
2           [281325]
3    [30621, 304743]
4            [60570]
5            [63525]
Name: movie_id, dtype: object

In [11]:
Directors

,id,first_name,last_name
0,1,Todd,1
1,2,Les,12 Poissons
2,3,Lejaren,a'Hiller
3,4,Nian,A
4,5,Khairiya,A-Mansour
...,...,...,...
86875,88797,Yusuf,Ünal
86876,88798,Ahmet,Ündag
86877,88799,Idil,Üner
86878,88800,Yüksel,Ünsal


,director_id,movie_id
163516,38057,0
257818,60728,1
327612,77912,1
41291,9970,2
303093,71885,5
...,...,...
317133,75198,412317
182167,42605,412317
3040,890,412318
339033,80751,412319


In [12]:
A = []
for idd in tqdm(Directors.id.unique()):
    temp = Directors_genres.loc[Directors_genres["director_id"]==idd]
    val = json.loads(temp[["genre","prob"]].to_json(orient='records'))
    B = {}
    if(val):
        B = {"id":int(idd),"first_name":str(Directors[Directors["id"]==idd]["first_name"].iloc[0]),
             "last_name":str(Directors[Directors["id"]==idd]["last_name"].iloc[0]), "genres":val}
    else:
        B = {"id":int(idd),"first_name":str(Directors[Directors["id"]==idd]["first_name"].iloc[0]),
             "last_name":str(Directors[Directors["id"]==idd]["last_name"].iloc[0]), "genres":list()}
    B["director_of"] = []
    if(idd in C.index):
        B["director_of"] = C.loc[idd]
    A.append(B)

100%|██████████| 86880/86880 [01:53<00:00, 763.40it/s]


In [23]:
df_test = pd.DataFrame(A)
df_test["len_list"]= df_test["director_of"].apply(lambda x :len(x))
df_test.head()

,id,first_name,last_name,genres,director_of,len_list
0,1,Todd,1,[],[378879],1
1,2,Les,12 Poissons,"[{'genre': 'Short', 'prob': 1.0}]",[281325],1
2,3,Lejaren,a'Hiller,"[{'genre': 'Drama', 'prob': 1.0}]","[30621, 304743]",2
3,4,Nian,A,[],[60570],1
4,5,Khairiya,A-Mansour,"[{'genre': 'Documentary', 'prob': 1.0}]",[63525],1


In [24]:
df_test.sort_values("len_list",ascending=False)

,id,first_name,last_name,genres,director_of,len_list
24545,25116,Dave,Fleischer,"[{'genre': 'Action', 'prob': 0.00162075}, {'ge...","[124, 3404, 5494, 5566, 6555, 7090, 7908, 8970...",616
55293,56530,Georges,Méliès,"[{'genre': 'Sci-Fi', 'prob': 0.0106383}, {'gen...","[1697, 5896, 7442, 7530, 8238, 10162, 10526, 1...",554
29897,30570,D.W.,Griffith,"[{'genre': 'Sci-Fi', 'prob': 0.00199203}, {'ge...","[5083, 6600, 6810, 6930, 6982, 7397, 7400, 785...",530
1917,1958,Gilbert M. 'Broncho Billy',Anderson,"[{'genre': 'Western', 'prob': 0.769679}, {'gen...","[5517, 5842, 11047, 11048, 11049, 11051, 11052...",360
24019,24576,Louis,Feuillade,"[{'genre': 'Mystery', 'prob': 0.0285714}, {'ge...","[5406, 8278, 8448, 8851, 15011, 15259, 16410, ...",345
...,...,...,...,...,...,...
79924,81688,Alain,Van Damme,"[{'genre': 'Drama', 'prob': 1.0}]",[],0
19487,19925,Gulab,Dhanwani,[],[],0
9498,9685,Sharon,Bridgeman,"[{'genre': 'Horror', 'prob': 1.0}, {'genre': '...",[],0
5941,6051,Silvano,Beltrame,"[{'genre': 'Documentary', 'prob': 1.0}]",[],0


In [27]:
with open('Structure2\DirectorFinal.json', 'w') as f:
    json.dump(A, f)

# End